In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# 读取数据
train = pd.read_csv('.\\data\\first_round_training_data.csv')
test = pd.read_csv('.\\data\\first_round_testing_data.csv')

In [3]:
# 特征列表
features = ["Parameter1","Parameter2","Parameter3","Parameter4","Parameter5","Parameter6","Parameter7","Parameter8","Parameter9","Parameter10"]

In [4]:
# 标签转化
def quality_encoder(x):
    return {'Excellent':0,'Good':1,'Pass':2,'Fail':3}[x]

train['label'] = train.Quality_label.apply(quality_encoder)
train['label_Excellent'] = 1*(train['label'] == 0)
train['label_Good'] = 1*(train['label'] == 1)
train['label_Pass'] = 1*(train['label'] == 2)
train['label_Fail'] = 1*(train['label'] == 3)

In [5]:
train

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,...,Attribute7,Attribute8,Attribute9,Attribute10,Quality_label,label,label_Excellent,label_Good,label_Pass,label_Fail
0,0.001660,0.591013,147.608373,38.186345,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385,...,177.243120,338.729256,2.021704,0.079526,Pass,2,0,0,1,0
1,1.601749,0.015052,0.035864,51.130326,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,...,3.723330,15.376190,0.986973,4.634376,Fail,3,0,0,0,1
2,0.098039,69.233685,0.080920,0.112265,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385,...,0.064196,0.576302,33.875790,1.813727,Fail,3,0,0,0,1
3,18.181860,0.047325,0.018061,1.098102,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,...,0.029777,0.246726,27.117165,0.081819,Fail,3,0,0,0,1
4,0.012085,0.008749,0.005509,524.327396,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,...,0.055053,0.014725,13.569707,18.138496,Fail,3,0,0,0,1
5,0.004062,14.556483,0.786945,0.010545,0.000525,0.001623,2286.523413,0.035407,0.593081,1.010385,...,0.576380,123.057489,16.133884,0.598517,Good,1,0,1,0,0
6,0.438449,1.232559,2.882699,0.610757,1.600654,0.464037,0.600827,17.850021,0.051850,0.010192,...,0.035493,0.146690,41.285376,5.985572,Good,1,0,1,0,0
7,48159.917401,0.002987,14.863813,0.063287,1.434060,0.314162,0.600827,17.850021,0.051850,0.010192,...,0.615609,3.321156,0.013034,0.045039,Good,1,0,1,0,0
8,1.456601,52.381578,20.625283,366.074831,1.434060,0.258497,0.600827,17.850021,0.051850,0.010192,...,0.029777,0.007033,3.397996,20.322889,Pass,2,0,0,1,0
9,0.000109,4.784654,31.916672,20.672388,1.284806,0.175007,0.600827,17.850021,0.051850,0.010192,...,1.356619,1.081445,0.056060,0.413597,Good,1,0,1,0,0


In [6]:
for feature in features:
    train[feature] = np.log1p(train[feature])
    test[feature] = np.log1p(test[feature])

train = train[train.Parameter1<=12]
train = train[train.Parameter2<=14]
train = train[train.Parameter3<=12]
train = train[train.Parameter5<=4]
train = train[train.Parameter8<=8]
train = train[train.Parameter9<=10]

In [ ]:
train['param7_2'] = train.Parameter7 * train.Parameter7
train['param8_2'] = train.Parameter8 * train.Parameter8
train['param10_2'] = train.Parameter10 * train.Parameter10
train['param5_2'] = train.Parameter5 * train.Parameter5
train['param6_2'] = train.Parameter6 * train.Parameter6

In [7]:
# 本地多分类准确率评估
local_train = train.iloc[:5000,:].copy()
local_valid = train.iloc[5000:,:].copy()

features = ["Parameter1","Parameter2","Parameter3","Parameter4",
            "Parameter6","Parameter7","Parameter8","Parameter9",
            "Parameter10"]

model = GradientBoostingClassifier(max_depth=3,learning_rate=0.1,n_estimators=100,random_state=666)
model.fit(local_train.loc[:,features],local_train.label)

local_valid['prediction'] = model.predict(local_valid.loc[:,features])
(local_valid.label == local_valid.prediction).mean()

0.46903553299492384

In [8]:
local_valid

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,...,Attribute8,Attribute9,Attribute10,Quality_label,label,label_Excellent,label_Good,label_Pass,label_Fail,prediction
5013,0.036327,1.786738,0.012428,3.541304,2.134902,1.925209,1.216109,2.747501,0.050550,2.403343,...,0.625612,9.364729,2.410085,Pass,2,0,0,1,0,2
5014,9.893912,4.593559,0.045811,0.004451,2.232420,1.925209,1.216109,2.747501,0.050550,2.403343,...,0.008998,0.147046,0.099833,Pass,2,0,0,1,0,2
5015,1.682542,1.137094,1.128348,0.017988,2.134902,1.925209,1.216109,2.747501,0.050550,2.403343,...,0.034395,9.839475,5.985572,Pass,2,0,0,1,0,2
5016,1.982482,0.021546,1.335890,0.497633,2.331095,2.443510,0.470520,2.936514,2.052066,6.250357,...,20.777039,7.684019,61.585865,Fail,3,0,0,0,1,2
5017,0.990001,4.196552,0.486523,2.541988,0.427443,0.447224,0.037761,2.936514,0.050550,5.595505,...,3.808151,128.753886,10.568820,Good,1,0,1,0,0,0
5018,1.059582,3.787676,2.009259,0.714600,0.889561,0.273199,0.470520,2.936514,0.050550,0.019463,...,4.022390,1.877165,0.320231,Good,1,0,1,0,0,1
5019,0.324752,0.033251,0.755918,0.984549,0.889561,0.323398,0.470520,2.936514,0.050550,0.019463,...,0.943148,0.009221,0.013265,Good,1,0,1,0,0,1
5020,3.001100,4.644404,7.480978,5.751357,0.955763,0.323398,0.470520,2.936514,0.050550,0.019463,...,0.260606,0.010963,0.018135,Good,1,0,1,0,0,1
5021,2.039874,0.660315,0.428757,0.048840,0.889561,0.192844,0.470520,2.936514,0.050550,0.019463,...,0.010897,0.006056,0.091672,Pass,2,0,0,1,0,1
5022,0.990302,0.000048,2.566307,5.323379,0.955763,0.381198,0.470520,2.936514,0.050550,0.019463,...,0.010317,0.308747,0.019749,Pass,2,0,0,1,0,1


In [9]:
# 线上提交
model = GradientBoostingClassifier(max_depth=3,learning_rate=0.1,n_estimators=100,random_state=666)
model.fit(train.loc[:,features],train.label)

test['prediction'] = model.predict(test.loc[:,features])
test['prob_Excellent'] = 0.0
test['prob_Good'] = 0.0
test['prob_Pass'] = 0.0
test['prob_Fail'] = 0.0
test.loc[:,['prob_Excellent','prob_Good','prob_Pass','prob_Fail']] = model.predict_proba(test.loc[:,features])

In [10]:
test.tail(100)

,Group,Parameter1,Parameter10,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,prediction,prob_Excellent,prob_Good,prob_Pass,prob_Fail
5900,118,0.000939,2.403343,0.649447,0.377107,1.025523,3.046845,2.443510,0.141551,0.034795,0.465670,2,0.079981,0.115380,0.760870,0.043768
5901,118,0.009746,2.403343,3.261623,3.071943,14.129487,2.331095,2.805111,0.141551,0.034795,0.465670,2,0.182415,0.060589,0.498301,0.258695
5902,118,2.652059,2.403343,3.125568,0.047589,7.152962,3.046845,2.443510,0.141551,0.034795,0.465670,2,0.093278,0.123792,0.728784,0.054145
5903,118,1.746904,2.403343,1.158065,1.727986,0.086635,2.331095,2.805111,0.141551,0.034795,0.465670,2,0.086343,0.109324,0.754205,0.050127
5904,118,0.015616,2.403343,0.017934,0.763712,0.989770,2.134902,2.443510,0.141551,0.034795,0.465670,2,0.079774,0.109012,0.761772,0.049443
5905,118,0.018728,2.403343,0.191746,0.257608,2.993477,2.942446,2.989361,0.141551,0.034795,0.465670,2,0.083629,0.100546,0.764013,0.051812
5906,118,1.688060,2.403343,0.065906,3.163983,0.000001,2.134902,2.267014,0.141551,0.034795,0.465670,2,0.074533,0.119158,0.762589,0.043720
5907,118,14.140331,2.403343,1.307960,0.281969,7.702266,2.038643,2.989361,0.141551,0.034795,0.465670,0,0.655766,0.041811,0.279941,0.022482
5908,118,2.154223,2.403343,0.844803,0.290084,4.939650,2.942446,2.989361,0.141551,0.034795,0.465670,2,0.093312,0.108236,0.744840,0.053613
5909,118,3.296509,2.403343,0.922438,0.053627,1.130623,2.038643,2.989361,0.141551,0.034795,0.465670,2,0.092053,0.106106,0.749682,0.052159


In [11]:
# 提交用
prediction = test.groupby(['Group'],as_index=False)['prob_Excellent','prob_Good','prob_Pass','prob_Fail'].mean()
prediction.columns = ['Group','Excellent ratio','Good ratio','Pass ratio','Fail ratio']
prediction.to_csv('submission12.csv',index=False)